<a href="https://colab.research.google.com/github/DeepSmeag/ML_Colab/blob/main/AND%2BXORperceptrons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/DeepSmeag/ML_Colab

Cloning into 'ML_Colab'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 47 (delta 19), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (47/47), done.


First we load the project so we get access to the files. Then we read the files of interest

In [ ]:
def read_file(file_path):
  dataset=[]
  with open(file_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
      line = line.strip()
      parts = line.split(',')
      x0,x1,y1,y2 = float(parts[0]), float(parts[1]), float(parts[2]), float(parts[3])
      dataset.append((x0,x1,y1,y2))
    return dataset

dataset = read_file('ML_Colab/datasetAND+XOR.csv')
"""it could also work with numpy and/or pandas to read the whole csv file at the
same time, probably more efficient too
"""


We now have our dataset ready for use, next let's go and make the vectors for use with our network

In [10]:
import numpy as np
import csv

In [6]:
#first layer is input, meaning we take the first 2 columns of the dataset
num_trains = len(dataset)
X = np.ones((num_trains,1)) 
#now we add the bias to every training instance
X = np.append(X, np.array(dataset)[:,0:2], axis=1)
# now we take the outputs
Y = np.array(dataset)[:,2:4]

In [13]:
#now we create Numpy arrays so that we have our weights
#the first layer
W1 = np.random.uniform(-1,1,(3,3))
#the second
W2 = np.random.uniform(-1,1,(4,2))
print("W1:\n",W1,"\n")
print("W2:\n",W2,"\n")

W1:
 [[-0.1485472   0.34717526  0.87401992]
 [ 0.25400568 -0.67257853 -0.13334229]
 [-0.91350926 -0.13719468 -0.29667054]] 

W2:
 [[-0.32814207  0.35365553]
 [ 0.92442804 -0.90134961]
 [ 0.32653827  0.09339608]
 [ 0.38228571 -0.23784318]] 



In [11]:
#Now we're getting to the fun stuff
#First, defining the necessary functions

def sigmoid(x):
  return 1/(1 + np.exp(-x))

#Forward Propagation
def forward_prop():
  #calculating the steps
  z2 = np.matmul(X, W1)
  a2 = sigmoid(z2)
  a2 = np.insert(a2, 0, np.ones((num_trains,)), axis = 1)
  z3 = np.matmul(a2, W2)
  y_hat = sigmoid(z3)
  return z2, a2, z3, y_hat


In [ ]:
#Ok, so that was forward propagation
#Now, the NN should also be able to learn, so we have to implement Backprop
#Since we're guessing at best at the moment as to the correct outputs
#We have to define several functions for Backprop to work
#These include the partial derivatives with which we'll update the weights matrices W1 and W2
#And also the backprop itself which will go over epochs or whatever we may call it

# matrices sizes
# X = num_trains x 3
# W1 = 3 x 3
# Z2 = num_trains x 3
# A2 = num_trains x 4
# W2 = 4 x 2
# Z3 = num_trains x 2
# A3 = Y_hat = num_trains x 2


def sigmoid_gradient(x):
  return sigmoid(x)*(1-sigmoid(x))
def derivative(z2, a2, z3, y_hat):
  dz3 = sigmoid_gradient(z3) # num_trains x 2
  dw2 = np.matmul(a2.transpose(), dz3) # 4 x 2
  dz2 = sigmoid_gradient(z2) # num_trains x 3
  temp1 = np.matmul(dz3, W2.transpose()) # num_trains x 4
  temp1 = np.delete(temp1, 3, 1) # drop last column because it's for bias, num_trains x 3
  temp2 = np.multiply(temp1, dz2) # num_trains x 3
  dw1 = np.matmul(temp2.transpose(), X) # 3 x 3
  return dw1, dw2

def backprop(epochs, learning_rate, W1, W2):
  for i in range(epochs):
    print("epoch: ", i+1, "\n")
    z2, a2, z3, y_hat = forward_prop()
    dw1, dw2 = derivative(z2, a2, z3, y_hat)
    W1 = W1 - learning_rate * dw1
    W2 = W2 - learning_rate * dw2
  with open("ML_Colab/AND+XORmodelW1.csv", 'w') as fw1:
    write = csv.writer(fw1)
    write.writerow(W1)
  with open("ML_Colab/AND+XORmodelW2.csv", 'w') as fw2:
    write = csv.writer(fw2)
    write.writerow(W2)

backprop(50, 0.0001, W1, W2)